Homework can be done in pairs, but each person needs to submit a the python book.


# 1.	In the last layer of a DL network or logistic regression with multiple outputs, what’s the difference between using a softmax activation and a sigmoid?

Softmax is used for multi-class classifaction. The softmax will return the output with a probability, where the sum of ALL outputs will add up to 1.
Sigmoid is used for multi-label classification. The sigmoid will return the output with a probability, but without considering other outputs. Does not have to add up to 1.

# 2.	What’s the advantage of using RELU vs sigmoid for intermediate layers? What’s the disadvantage?
Relu tackles the vanishing gradient problem since its gradient is 0 or 1. 
Sigmoid can have the vanishing gradient problem as it can stall as it reachs values close to 0.

Relu is much faster computationally, while sigmoid is more expensive due to the exponential function.

# 3.	Some people have suggested an all convolutional network instead of using Dense layers at the end. What’s the advantage of that?

Fewer parameters to compute. Less chances of overfitting. Computationally efficient.

# 4.	What’s L1 and L2 regularization?

Loss functions
L1 can perform feature selection. Good for only choose a few features.
L2 uses the square of the weights, which can shrink the weights to 0 (as they are <1). prevents the model from changing drastically from small changes.

# 5. What's the difference of using MSE or crossentropy as a loss function? What's Hubber loss function?

MSE used typically for regression. Can be sensitive to outliers between the predicted values and actual values.

cross-entropy looks tat the probabilities between the true and predicted distrubution. Used for binary classifaction and multi classifcation.

Huberloss combines MSE and Mean Absolute Error, making it less sensitive to outliers. It makes a comparison between the errors of MSE and a set threshold, choosing between MSE or MAE.

# 6.	Review the two papers of the module and summarize them.

**Convolutional Neural Networks for Sentence Classification**
Yoon Kim

Paper focuses on CNN with one layer convolution using word vectors utilizing word2vec. The model practically represents the sentences as a matrix, which is fed through a convolutional layer where each filter extracts one feature. Afterwards, its fed to a max-over-time pooling over the geature map to take the max value. Lastly, the model is dropped into a fully connected layter with dropout with a softmax output for regularization. They performed their model against benchmarks 
* (MR)Movie reviews
* (SST-1 and SST-2)Stanford Sentiment Treebank
* (Subj)Subjectivity dataset
* (TREC) question dataset
* (CR) Customer reviews
* (MPQA) Opinion polarity detection

Kim and the team utilized four different model variations  while utilizing word2vec. They were able to show that their CNN_rand model didn't perform as well, while their other models did, setting out to prove that a simple CNN with one layer of convolution performs well.

**A Convolutional Neural Network for Modelling Sentences**
Nal Kalchbrenner Edward Grefenstette Phil Blussom

Paper introducess DCNN (Dynamic Convolutional Neural Network). It's utilized for semantic modelling of sentences. 

The model is as follows:
* K-max pooling - which allows the sentences to be modeled for wide convolutional layers. Allows the pooling of the most prominent features that may be far apart.
* Folding - which practically halves the number of the rows by summing up every two rows.
* Wide convolution - utilizing one-dimensional convolution
* Dynamic k-max pooling - where the value of k is determined by the length of the sentences and the depth of the network.

They ran numerous expirements, where specifically on the Twitter sentiment data set, DCNN outperformed the other models.

For the next two problems, you will use the following code packages (you will implement the solution using SKLearn Logistic Regression and PyTorch. Present your results in a tabular format with 3 result columns: Naive Bayes, SKLearn Logic Regression, PyTorch Logistic Regression.

Note: for multiclass, you will use logits (the output of the linear layer before softmax). Look at the documentation for ```torch.nn.CrossEntropyLoss()```.

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
import torch

In [35]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Model(torch.nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

x_train_pt = torch.tensor(x_train_m[:1000],dtype=torch.float32)
y_train_pt = torch.tensor(y_train[:1000],dtype=torch.float32).unsqueeze(1)
x_test_pt = torch.tensor(x_test_m[:1000],dtype=torch.float32)
y_test_pt = torch.tensor(y_test[:1000],dtype=torch.float32).unsqueeze(1)

train_data = TensorDataset(x_train_pt, y_train_pt)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

model = Model(x_train_m.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.BCEWithLogitsLoss()

max_iter = 10
log_interval = 1

for iter_num in range(max_iter):
    model.train()
    epoch_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        p_train = model(inputs)
        
        # Compute loss
        loss_train = loss_fn(p_train, labels)
        
        # backprop step
        loss_train.backward()

        # update weights
        optimizer.step()
        
        epoch_loss += loss_train.item()
    
    # Log the training loss at intervals
    if iter_num % log_interval == 0:
        model.eval()
        with torch.no_grad():
            p_test = model(x_test_pt)
            loss_test = loss_fn(p_test, y_test_pt).item()
        
        avg_epoch_loss = epoch_loss / len(train_loader)
        print(f'Iteration {iter_num}: train loss = {avg_epoch_loss:.4f}, test loss = {loss_test:.4f}')

Iteration 0: train loss = 0.6931, test loss = 0.6928
Iteration 1: train loss = 0.6921, test loss = 0.6925
Iteration 2: train loss = 0.6912, test loss = 0.6921
Iteration 3: train loss = 0.6903, test loss = 0.6918
Iteration 4: train loss = 0.6894, test loss = 0.6914
Iteration 5: train loss = 0.6885, test loss = 0.6910
Iteration 6: train loss = 0.6876, test loss = 0.6907
Iteration 7: train loss = 0.6867, test loss = 0.6903
Iteration 8: train loss = 0.6858, test loss = 0.6900
Iteration 9: train loss = 0.6849, test loss = 0.6896


In [36]:
with torch.no_grad():
    train_outputs = model(x_train_pt)
    test_outputs = model(x_test_pt)
    
    train_accuracy = ((train_outputs > 0) == y_train_pt).float().mean().item()
    test_accuracy = ((test_outputs > 0) == y_test_pt).float().mean().item()

print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Train Accuracy: 0.9460
Test Accuracy: 0.7250


## 6.1.	Using the same data from HW-4, exercise 3, create a Deep Learning classifier to perform sentiment analysis of the movie reviews. Perform the same comparison you did in HW-4 w.r.t. Naïve Bayes.  You will present the result in a table with accuracy for columns `Naïve Bayes`, `LogisticRegression` and `Deep Learning`.

In [4]:
from keras.datasets import imdb
from keras.preprocessing import sequence

start_char = 1
oov_char = 2
index_from = 3
maxlen = 5000
# Retrieve the training sequences.
(x_train, y_train), (x_test, y_test) = imdb.load_data(
    start_char=start_char, oov_char=oov_char, index_from=index_from, maxlen=maxlen
)
word_index = imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
# And add `index_from` to indices to sync with `x_train`
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[0] = "[PAD]"
inverted_word_index[index_from] = "[NOP]"
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"
# Decode the first sequence in the dataset
x_train_corpus = [" ".join([inverted_word_index[j] for j in x_train[i]]) for i in range(len(x_train))]
x_test_corpus = [" ".join([inverted_word_index[j] for j in x_test[i]]) for i in range(len(x_test))]
decoded_sequence = x_train_corpus[0]
decoded_sequence

"[START] this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh

In [7]:
vocabulary = [inverted_word_index[i] for i in range(len(inverted_word_index))]
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary, lowercase=False)), ('tfid', TfidfTransformer())]).fit(x_train_corpus)
x_train_m = pipe.transform(x_train_corpus).toarray()
x_train_m.shape

(25000, 88588)

In [9]:
clf = LogisticRegression(max_iter=10000).fit(x_train_m, y_train)

In [10]:
x_test_m = pipe.transform(x_test_corpus).toarray()
clf.score(x_train_m, y_train), clf.score(x_test_m, y_test)

(0.93304, 0.88312)

In [11]:
list(zip(clf.predict_proba(x_test_m), y_test))

[(array([0.80302108, 0.19697892]), 0),
 (array([0.0341813, 0.9658187]), 1),
 (array([0.30628318, 0.69371682]), 1),
 (array([0.55254275, 0.44745725]), 0),
 (array([0.11605194, 0.88394806]), 1),
 (array([0.41074334, 0.58925666]), 1),
 (array([0.22513766, 0.77486234]), 1),
 (array([0.72476545, 0.27523455]), 0),
 (array([0.1522964, 0.8477036]), 0),
 (array([0.21737369, 0.78262631]), 1),
 (array([0.31806929, 0.68193071]), 1),
 (array([0.90207633, 0.09792367]), 0),
 (array([0.87058069, 0.12941931]), 0),
 (array([0.76029225, 0.23970775]), 0),
 (array([0.03887468, 0.96112532]), 1),
 (array([0.99114453, 0.00885547]), 0),
 (array([0.07177949, 0.92822051]), 1),
 (array([0.42042295, 0.57957705]), 0),
 (array([0.79430436, 0.20569564]), 0),
 (array([0.83691742, 0.16308258]), 0),
 (array([0.06087934, 0.93912066]), 1),
 (array([0.04972793, 0.95027207]), 1),
 (array([0.58740323, 0.41259677]), 1),
 (array([0.10588575, 0.89411425]), 1),
 (array([0.22399681, 0.77600319]), 1),
 (array([0.33913322, 0.660866

In [13]:
#Linear regression
accuracy_lr_train = clf.score(x_train_m, y_train)
accuracy_lr_test = clf.score(x_test_m, y_test)


print(f'Logistic Regression Train Accuracy: {accuracy_lr_train:.4f}')
print(f'Logistic Regression Test Accuracy: {accuracy_lr_test:.4f}')

Logistic Regression Train Accuracy: 0.9330
Logistic Regression Test Accuracy: 0.8831


In [15]:
# Train Naive Bayes model
clf_nb = MultinomialNB().fit(pipe.transform(x_train_corpus), y_train)

accuracy_nb_train = clf_nb.score(pipe.transform(x_train_corpus), y_train)
accuracy_nb_test = clf_nb.score(pipe.transform(x_test_corpus), y_test)

print(f'Naive Bayes Train Accuracy: {accuracy_nb_train:.4f}')
print(f'Naive Bayes Test Accuracy: {accuracy_nb_test:.4f}')

Naive Bayes Train Accuracy: 0.9086
Naive Bayes Test Accuracy: 0.8298


In [43]:
import pandas as pd

naive_bayes_train_accuracy = 0.9086
naive_bayes_test_accuracy = 0.8298
logistic_regression_train_accuracy = 0.9330
logistic_regression_test_accuracy = 0.8831
deep_learning_train_accuracy = 0.9460
deep_learning_test_accuracy = 0.7250

results = {
    "Accuracy": ["Train", "Test"],
    "Naive Bayes": [
        naive_bayes_train_accuracy, 
        naive_bayes_test_accuracy
    ],
    "Regression": [
        logistic_regression_train_accuracy,
        logistic_regression_test_accuracy
    ],
    "PyTorch": [
        deep_learning_train_accuracy,
        deep_learning_test_accuracy
    ],
}

results_df = pd.DataFrame(results)

print(results_df)

  Accuracy  Naive Bayes  Regression  PyTorch
0    Train       0.9086      0.9330    0.946
1     Test       0.8298      0.8831    0.725


## 6.2.	Using the same data from HW-4, exercise 4, create a DL network using Conv1D’s to perform document classification.  Do not use Embeddings yet. Try at least two network architectures. You will present the result in a table with accuracy for columns `Naïve Bayes`, `XGBoost` and `Deep Learning #1` and `Deep Learning #2`. 

Instead of using TFIDF or count vectors, we will use `sentence transformers`, which compute an embedding vector for the sentence.

In [1]:
import torch

In [2]:
from keras.datasets import reuters
from keras.utils import to_categorical
from sentence_transformers import SentenceTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load dataset
(train_data, train_labels), (test_data, test_labels) = reuters.load_data()
w2i = reuters.get_word_index()
i2w = dict([(value, key) for (key, value) in w2i.items()])

# Convert to text
train_data_text = [' '.join([i2w.get(i - 3, '?') for i in s]) for s in train_data]
test_data_text = [' '.join([i2w.get(i - 3, '?') for i in s]) for s in test_data]

# SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda')
x_train_e = model.encode(train_data_text)
x_test_e = model.encode(test_data_text)

x_train_e.shape, x_test_e.shape  # Ensure the shapes are correct

num_classes = max(train_labels) + 1
y_train = to_categorical(train_labels, num_classes)
y_test = to_categorical(test_labels, num_classes)


C:\Users\Owner\anaconda3\envs\torch\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Owner\anaconda3\envs\torch\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
# Train Naive Bayes model
nb_model = GaussianNB()
nb_model.fit(x_train_e, train_labels)
nb_pred = nb_model.predict(x_test_e)

# accuracy
accuracy_nb = accuracy_score(test_labels, nb_pred)

print(f"Naive Bayes Accuracy: {accuracy_nb:.4f}")


Naive Bayes Accuracy: 0.7471


In [12]:
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense

# copied from last quarter
model1 = Sequential([
    Conv1D(128, 5, activation='relu', input_shape=(x_train_e.shape[1], 1)),
    GlobalMaxPooling1D(),
    Dense(num_classes, activation='softmax')
])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 380, 128)          768       
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 46)                5934      
                                                                 
Total params: 6,702
Trainable params: 6,702
Non-trainable params: 0
_________________________________________________________________


In [13]:

#copied from last quarter
model2 = Sequential([
    Conv1D(64, 5, activation='relu', input_shape=(x_train_e.shape[1], 1)),
    Conv1D(64, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 380, 64)           384       
                                                                 
 conv1d_2 (Conv1D)           (None, 376, 64)           20544     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 46)                5934      
                                                                 
Total params: 35,182
Trainable params: 35,182
Non-trainable params: 0
__________________________________________________

In [14]:
# Reshape input
x_train_e = x_train_e[..., None]
x_test_e = x_test_e[..., None]

# Train the models
history1 = model1.fit(x_train_e, y_train, epochs=10, batch_size=32, validation_split=0.2)
history2 = model2.fit(x_train_e, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the models
loss1, accuracy1 = model1.evaluate(x_test_e, y_test)
loss2, accuracy2 = model2.evaluate(x_test_e, y_test)

print(f"Deep Learning #1 Accuracy: {accuracy1:.4f}")
print(f"Deep Learning #2 Accuracy: {accuracy2:.4f}")


Epoch 1/10
225/225 [==============================] - 9s 37ms/step - loss: 2.7790 - accuracy: 0.3445 - val_loss: 2.3992 - val_accuracy: 0.3450
Epoch 2/10
225/225 [==============================] - 6s 28ms/step - loss: 2.3981 - accuracy: 0.3534 - val_loss: 2.3940 - val_accuracy: 0.3450
Epoch 3/10
225/225 [==============================] - 6s 28ms/step - loss: 2.3894 - accuracy: 0.3534 - val_loss: 2.3854 - val_accuracy: 0.3450
Epoch 4/10
225/225 [==============================] - 5s 21ms/step - loss: 2.3831 - accuracy: 0.3534 - val_loss: 2.3818 - val_accuracy: 0.3450
Epoch 5/10
225/225 [==============================] - 6s 29ms/step - loss: 2.3755 - accuracy: 0.3534 - val_loss: 2.3749 - val_accuracy: 0.3450
Epoch 6/10
225/225 [==============================] - 9s 38ms/step - loss: 2.3667 - accuracy: 0.3534 - val_loss: 2.3614 - val_accuracy: 0.3450
Epoch 7/10
225/225 [==============================] - 7s 33ms/step - loss: 2.3568 - accuracy: 0.3534 - val_loss: 2.3530 - val_accuracy: 0.3450

In [3]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Train
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(x_train_e, train_labels)

xgb_pred = xgb_model.predict(x_test_e)

# Evaluate
accuracy_xgb = accuracy_score(test_labels, xgb_pred)
print(f"XGBoost Accuracy: {accuracy_xgb:.4f}")


XGBoost Accuracy: 0.7734


In [1]:
accuracy_xgb = 0.90  
accuracy_nb = 0.7471  
accuracy1 = 0.3629    
accuracy2 = 0.5071    

# Results table
results = {
    "Model": ["Naïve Bayes", "XGBoost", "Deep Learning #1", "Deep Learning #2"],
    "Accuracy": [accuracy_nb, accuracy_xgb, accuracy1, accuracy2]
}

import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)


              Model  Accuracy
0       Naïve Bayes    0.7471
1           XGBoost    0.9000
2  Deep Learning #1    0.3629
3  Deep Learning #2    0.5071
